# Build Classification Model

In [80]:
import pandas as pd
cuisines_df = pd.read_csv("../data/cleaned_cuisines.csv")
cuisines_df.head()

,Unnamed: 0,cuisine,almond,angelica,anise,anise_seed,apple,apple_brandy,apricot,armagnac,...,whiskey,white_bread,white_wine,whole_grain_wheat_flour,wine,wood,yam,yeast,yogurt,zucchini
0,0,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,indian,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [81]:
cuisines_label_df = cuisines_df['cuisine']
cuisines_label_df.head()

0    indian
1    indian
2    indian
3    indian
4    indian
Name: cuisine, dtype: object

In [82]:
cuisines_feature_df = cuisines_df.drop(['Unnamed: 0', 'cuisine'], axis=1)
cuisines_feature_df.head()

,almond,angelica,anise,anise_seed,apple,apple_brandy,apricot,armagnac,artemisia,artichoke,...,whiskey,white_bread,white_wine,whole_grain_wheat_flour,wine,wood,yam,yeast,yogurt,zucchini
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [83]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score,precision_score,confusion_matrix,classification_report, precision_recall_curve
import numpy as np

In [84]:
X_train, X_test, y_train, y_test = train_test_split(cuisines_feature_df, cuisines_label_df, test_size=0.3)

### SVC 支持向量分类

用这种方法您可以选择一个 kernel 去决定如何聚类标签。C 参数指的是“正则化”，它将参数的影响正则化。kernel 可以是其中的一项；这里我们将 kernel 设置为 linear 来使用线性 SVC。probability 默认为 false，这里我们将其设置为 true 来收集概率估计。我们还将 random_state 设置为 0 去打乱数据来获得概率。

C: C值越大，分类器就越不愿意允许分类错误（“离群点”）。如果C值太大，分类器就会竭尽全力地在训练数据上少犯错误，而实际上这是不可能/没有意义的，于是就造成过拟合。而C值过小时，分类器就会过于“不在乎”分类错误，于是分类性能就会较差。



In [85]:
C = 10
# 创建不同的分类器
classifiers = {
    'Linear SVC': SVC(kernel='linear', C=C, probability=True,random_state=0),
    'KNN classifier': KNeighborsClassifier(C),
    'SVC': SVC(),
    'RFST': RandomForestClassifier(n_estimators=100),
    'ADA': AdaBoostClassifier(n_estimators=100)
}

#### 正则化
由于损失函数只考虑在训练集上的[经验风险](https://zh.wikipedia.org/wiki/经验风险最小化)，这种做法可能会导致[过拟合](https://zh.wikipedia.org/wiki/过拟合)。为了对抗过拟合，我们需要向损失函数中加入描述模型复杂程度的正则项Ω(F)，将经验风险最小化问题转化为结构风险最小化。

机器学习中常常提到的正则化到底是什么意思？ - 陶轻松的回答 - 知乎
https://www.zhihu.com/question/20924039/answer/131421690

In [86]:
n_classifiers = len(classifiers)

for index, (name, classifier) in enumerate(classifiers.items()):
    classifier.fit(X_train.values, np.ravel(y_train))

    y_pred = classifier.predict(X_test.values)
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy (train) for %s: %0.1f%% " % (name, accuracy * 100))
    print(classification_report(y_test,y_pred))

Accuracy (train) for Linear SVC: 79.0% 
              precision    recall  f1-score   support

     chinese       0.71      0.70      0.71       242
      indian       0.90      0.86      0.88       246
    japanese       0.73      0.76      0.75       220
      korean       0.82      0.74      0.78       238
        thai       0.78      0.87      0.83       253

    accuracy                           0.79      1199
   macro avg       0.79      0.79      0.79      1199
weighted avg       0.79      0.79      0.79      1199

Accuracy (train) for KNN classifier: 73.5% 
              precision    recall  f1-score   support

     chinese       0.69      0.69      0.69       242
      indian       0.87      0.80      0.83       246
    japanese       0.61      0.84      0.71       220
      korean       0.91      0.54      0.68       238
        thai       0.70      0.81      0.75       253

    accuracy                           0.73      1199
   macro avg       0.76      0.74      0.73    

### 其他分类器
随机森林是一种平均化方法，它建立了一个注入了随机性的“决策树森林”以避免过度拟合。n_estimators 参数设置了随机森林中树的数量。
> - 随机森林由多个决策树组成，每个决策树都是一个独立的分类器。在构建随机森林时，首先从原始数据集中随机选择一部分样本(有放回地抽样)，形成一个训练子集。然后使用这个子集来构建一个决策树模型，每个决策树都基于不同的子集构建。这样可以保证每个决策树都有一定的差异性。
> - 在构建决策树的过程中，随机森林采用了一些随机性的技巧。例如，在每个节点上，不是根据所有特征来选择最佳划分特征，而是从所有特征中随机选择一部分特征进行划分。这样做的目的是增加决策树的多样性，减少过拟合的可能性。
> - 当进行预测时，随机森林中的每个决策树都会对样本进行预测，然后通过投票或取平均值的方式来确定最终的预测结果。对于分类问题，采用多数投票的方式；对于回归问题，取平均值。

随机森林的优点
> - 可以处理高维数据，并且对缺失值和异常值具有较好的容错能力。
> - 能够估计特征的重要性，可以帮助我们理解数据集中的关键特征。
> - 不容易过拟合，因为每个决策树都是基于不同的子集构建的，并且通过投票或取平均值的方式来决定最终的预测结果。
> - 在处理大规模数据集时具有较好的计算性能，可以并行处理。

AdaBoost 用于解决二分类和多分类问题。它通过串行训练一系列弱分类器，并根据其表现调整实例的权重，从而构建一个强分类器。
> - 初始化训练集中每个实例的权重，使它们均匀分布。
> -  依次训练弱分类器，每个分类器都在当前样本权重下进行训练。训练过程中，弱分类器会根据分类错误率调整样本的权重，使得之前被错误分类的样本在下一轮训练中得到更多的关注。
> - 对于每个弱分类器，根据其分类错误率计算其权重，错误率低的分类器会获得更高的权重。
> - 将所有弱分类器组合成一个强分类器，通过加权投票或加权求和（权重取决于每个弱分类器的性能，性能越好的弱分类器会被赋予更高的权重。）的方式进行预测。

优点包括
> - 可以处理复杂的分类问题，具有较高的准确性。
> - 在训练过程中，重点关注错误分类的样本，从而提高整体分类性能。
> - 不容易过拟合，因为弱分类器简单且数量较多，可以在训练过程中进行调整和优化。
> - 在组合弱分类器时，能够有效利用特征的信息，提高整体分类性能。

缺点
> - 对噪声和异常值敏感，可能会导致错误的权重更新。
> - 对于高维数据或特征维度较大的数据集，需要更多的弱分类器和迭代次数。
> - 训练时间较长，因为需要串行训练多个弱分类器。



